In [1]:
import numpy as np
import os 
import matplotlib.pyplot as plt
import time
from sumo_rl import SumoEnvironment
import torch

In [2]:
print("SUMO HOME:", os.environ.get("SUMO_HOME"))

SUMO HOME: /opt/miniconda3/lib/python3.13/site-packages/sumo


In [3]:
# ===== Check the actions and the state space =====
# print("Action Space:", env.action_space)
# print("State Space:", env.observation_space)

In [ ]:
# ========= Simple simulation test of  SUMO-RL ===========

demo_trigger = input("Do you want to run the demo?")
if demo_trigger.lower() in ['y', 'yes']:
    print("Demo Starting....")
    exit()

    # File paths
    NET_FILE = 'single-intersection.net.xml'      # network file
    ROU_FILE = 'single-intersection-vertical.rou.xml'  # route file (traffic flows)

    # Initialize SUMO environment
    env = SumoEnvironment(
        net_file=NET_FILE,
        route_file=ROU_FILE,
        use_gui=False,                 # show SUMO GUI
        single_agent=True,            # one intersection controller
        reward_fn='diff-waiting-time',
        out_csv_name='outputs/traffic_log'
    )

    # Intialization
    obs, info = env.reset()
    avg_speed_per_road, avg_wait_time_per_road = np.zeros(4), np.zeros(4)
    avg_pressure, avg_total_speed = [],[]
    rewards, actions = [], []
    sim_time = 0.0

    for step in range(100):

        print(f"--- Step {step} / Time {sim_time:.1f} ---")

        # Take a random action
        action = env.action_space.sample()
        actions.append(action)

        # Recieve the env feedback
        obs, reward, done, truncated, info = env.step(action)

        # Current simulation time
        sim_time = env.sumo.simulation.getTime()

        # Get the vehicles IDs
        veh_ids = env.sumo.vehicle.getIDList()

        # Collisions Detection
        # collisions = env.sumo.simulation.getCollidingVehiclesIDList()
        # if len(collisions) > 0:
        #     print(f"Collisions: {collisions}")

        #     # Reset env after collision
        #     obs, info = env.reset()
        #     print("Environment reset.\n")
        #     break

        # Integrate information from the environment
        lane_counter = {}
        total_speed = 0.0
        intsec_pressure = 0.0

        for vid in veh_ids:
            speed = env.sumo.vehicle.getSpeed(vid)
            lane = env.sumo.vehicle.getLaneID(vid)
            # pos = env.sumo.vehicle.getPosition(vid)
            # print(f"{vid}: {speed:.2f} m/s on {edge} ({lane}), pos={pos}")
            total_speed += speed
            lane_counter[lane] =  lane_counter.get(lane,0) + 1
        
        avg_speed = total_speed/len(veh_ids) if len(veh_ids) > 0 else 0
        avg_total_speed.append(avg_speed)

        print(f"Average speed of all vehicles: {avg_speed:.2f} m/s")
        print(f"Vechicles per lane:{lane_counter}")


        # Arrived vehicles
        # arrived = env.sumo.simulation.getArrivedIDList()
        # if len(arrived) > 0:
        #     print(f"Arrived: {arrived}")

        # Red-Green Light States
        # for tls in env.ts_ids:
        state = env.sumo.trafficlight.getRedYellowGreenState(env.ts_ids[0])
        phase = env.sumo.trafficlight.getPhase(env.ts_ids[0])
        print(f"TraLight state=:{state} ")

        # Reward
        print(f"Step reward: {reward:.3f}\n")

        time.sleep(0.2)

        if done:
            obs, info = env.reset()
            print("Environment reset.\n")

    env.close()
    print("Demo Simulation finished!")
else:
    print("Demo skipped.")

Demo Starting....
 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 8ms, vehicles TOT 0 ACT 0 BUF 0)                      
 Retrying in 1 seconds
Could not connect to TraCI server at localhost:55986 [Errno 61] Connection refused
 Retrying in 1 seconds
Could not connect to TraCI server at localhost:55986 [Errno 61] Connection refused
 Retrying in 1 seconds
Could not connect to TraCI server at localhost:55986 [Errno 61] Connection refused
 Retrying in 1 seconds
Could not connect to TraCI server at localhost:55986 [Errno 61] Connection refused
 Retrying in 1 seconds
Could not connect to TraCI server at localhost:55986 [Errno 61] Connection refused
 Retrying in 1 seconds
Could not connect to TraCI server at localhost:55986 [Errno 61] Connection refused
 Retrying in 1 seconds
--- Step 0 / Time 0.0 ---
Average speed of all vehicles: 12.48 m/s
Vechicles per lane:{'e_t_0': 2, 'e_t_1': 1, 'n_t_1': 1, 'n_t_0': 2, 's_t_0': 2, 's_t_1': 1, 'w_t_0': 2, 'w_t_1': 1}
TraLight state=:GGrrrrGGrrr

: 

In [ ]:
from ppo import ActorCritic
from single_intersection import TrafficEnv

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# SUMO command (headless for speed)
sumo_cmd = [
    # "sumo-gui", # "sumo" or "sumo-gui""
    # "--start", # Uncomment this line while using the GUI for visualization 
    "--no-warnings", "true", # Uncomment this line to 
    "-n", "single-intersection.net.xml",
    "-r", "single-intersection-vertical.rou.xml",
    "--step-length", "1.0"
]

TLS_ID = "t"    

# File paths
NET_FILE = 'single-intersection.net.xml'      # network file
ROU_FILE = 'single-intersection-vertical.rou.xml'  # route file (traffic flows)

# Initialize SUMO environment
env = TrafficEnv(
    net_file=NET_FILE,
    route_file=ROU_FILE,
    use_gui=True,                 # show SUMO GUI
    single_agent=True            # one intersection controller
)

In [ ]:
def train_ppo(
    env=None,
    GAMMA = 0.99,
    GAE_LAMBDA = 0.95,
    CLIP_EPS = 0.2,
    LR = 3e-4,
    ENT_COEF = 0.01,
    VF_COEF = 0.5,
    MAX_GRAD_NORM = 0.5,
    N_STEPS = 2048, 
    N_EPOCHS = 10,        
    MINI_BATCH_SIZE = 256, 
    TOTAL_TIMESTEPS = 200_000,
    DEVICE=None
):
    

    if env is None:
        raise ValueError("Please provide a valid environment instance.")
    
    obs_dim = env.observation_space.shape[0]
    act_dim = env.action_space.n

    print("Observation dim:", obs_dim)
    print("Action dim:", act_dim)

    model = ActorCritic(obs_dim, act_dim).to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=LR)

    global_step = 0
    episode_returns = []
    current_ep_return = 0.0

    obs, info = env.reset()

    while global_step < TOTAL_TIMESTEPS:
        # 1) 采样一批 rollout
        batch = collect_rollout(env, model, N_STEPS)
        global_step += N_STEPS

        obs_arr = batch["obs"] 
        actions_arr = batch["actions"]
        old_logprobs_arr = batch["logprobs"]
        rewards_arr = batch["rewards"]
        dones_arr = batch["dones"]
        values_arr = batch["values"]
        next_value = batch["next_value"]

        # 2) 计算 GAE & returns
        advantages, returns = compute_gae(
            rewards_arr, values_arr, 
            dones_arr, next_value,
            gamma=GAMMA, lam=GAE_LAMBDA
        )

        # 标准化 advantage 可以加速训练
        advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)

        # 转成 tensor
        obs_t = torch.tensor(obs_arr, dtype=torch.float32, device=DEVICE)
        actions_t = torch.tensor(actions_arr, dtype=torch.int64, device=DEVICE)
        old_logprobs_t = torch.tensor(old_logprobs_arr, dtype=torch.float32, device=DEVICE)
        advantages_t = torch.tensor(advantages, dtype=torch.float32, device=DEVICE)
        returns_t = torch.tensor(returns, dtype=torch.float32, device=DEVICE)

        # 3) PPO 多 epoch 训练
        dataset_size = N_STEPS
        indices = np.arange(dataset_size)

        for epoch in range(N_EPOCHS):
            np.random.shuffle(indices)
            for start in range(0, dataset_size, MINI_BATCH_SIZE):
                end = start + MINI_BATCH_SIZE
                mb_idx = indices[start:end]

                mb_obs = obs_t[mb_idx]
                mb_actions = actions_t[mb_idx]
                mb_old_logprobs = old_logprobs_t[mb_idx]
                mb_advantages = advantages_t[mb_idx]
                mb_returns = returns_t[mb_idx]

                # 计算当前策略下 log_prob, entropy, value
                new_logprobs, entropy, values_pred = model.evaluate_actions(mb_obs, mb_actions)

                # ratio = π_θ(a|s) / π_θ_old(a|s)
                ratio = torch.exp(new_logprobs - mb_old_logprobs)

                # PPO clipped surrogate
                surr1 = ratio * mb_advantages
                surr2 = torch.clamp(ratio, 1.0 - CLIP_EPS, 1.0 + CLIP_EPS) * mb_advantages
                actor_loss = -torch.min(surr1, surr2).mean()

                # value function loss
                critic_loss = nn.MSELoss()(values_pred, mb_returns)

                # entropy 促进探索
                entropy_loss = -entropy.mean()

                loss = actor_loss + VF_COEF * critic_loss + ENT_COEF * entropy_loss

                optimizer.zero_grad()
                loss.backward()
                nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_NORM)
                optimizer.step()



        # 4) 打印训练信息（粗略统计一下最近一批的平均奖励）
        batch_mean_return = rewards_arr.mean() * N_STEPS  # 很粗略，仅为参考
        print(f"[Step {global_step}] recent mean reward per step: {rewards_arr.mean():.3f}")

    env.close()
    torch.save(model.state_dict(), "ppo_traffic_signal.pth")
    print("Training finished, model saved to ppo_traffic_signal.pth")


if __name__ == "__main__":
    train_ppo()
